## 역전파 학습법을 이용한 심층 신경망 학습

## Import modules

In [17]:
import time
import numpy as np

## 유틸리티 함수

In [18]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid 구현

In [19]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1

    def __call__(self, x):
        self.last_o = 1.0 / (1.0 + np.exp(-x))
        return self.last_o

    def grad(self):
        return self.last_o * (1.0 - self.last_o)

## Mean Squared Error 구현

In [20]:
class MeanSquaredError:
    def __init__(self):
        self.dh = 1
        self.last_diff = 1

    def __call__(self, h, y):
        self.last_diff = h - y
        return 1 / 2 * np.mean(np.square(self.last_diff))

    def grad(self):
        return self.last_diff

## Dense Layer 구현

In [21]:
class Dense:
    def __init__(self, W, b, a_obj):
        self.W = W
        self.b = b
        self.a = a_obj()
        
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W))
        
        self.last_x = np.zeros((self.W.shape[0]))
        self.last_h = np.zeros((self.W.shape[1]))
        

    def __call__(self, x):
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self): # dy/dh = W
        return self.W * self.a.grad()

    def grad_W(self, dh):
        grad = np.ones_like(self.W)
        grad_a = self.a.grad()
        for j in range(grad.shape[1]): # dy/dw = x
            grad[:, j] = dh[j] * grad_a[j] * self.last_x
        return grad

    def grad_b(self, dh): # dy/db = 1
        return dh * self.a.grad()

## 심층신경망 구현

In [22]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, input, output, activation=Sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(input, num_neuron)
        self.sequence.append(Dense(W, b, activation))

        # Hidden Layers
        for index in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Dense(W, b, activation))

        # Output Layer
        W, b = init_var(num_neuron, output)
        self.sequence.append(Dense(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)
        
        # back-prop loop
        for i in range(len(self.sequence) - 1, 0, -1):
            l1 = self.sequence[i]
            l0 = self.sequence[i - 1]
            
            l0.dh = _m(l0.grad(), l1.dh)
            l0.dW = l0.grad_W(l1.dh)
            l0.db = l0.grad_b(l1.dh)
        
        self.sequence.remove(loss_obj)

## 경사하강 학습법

In [23]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = loss_obj(network(x), y)  # Forward inference
    network.calc_gradient(loss_obj)  # Back-propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [24]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.11816118306543714
Epoch 1: Test loss 0.11682444197101738
Epoch 2: Test loss 0.11550274103112969
Epoch 3: Test loss 0.1141961220490419
Epoch 4: Test loss 0.11290461559357727
Epoch 5: Test loss 0.1116282413062084
Epoch 6: Test loss 0.1103670082210512
Epoch 7: Test loss 0.10912091509607633
Epoch 8: Test loss 0.10788995075389415
Epoch 9: Test loss 0.10667409443051486
Epoch 10: Test loss 0.10547331613054266
Epoch 11: Test loss 0.10428757698731925
Epoch 12: Test loss 0.1031168296265992
Epoch 13: Test loss 0.10196101853240645
Epoch 14: Test loss 0.10082008041379564
Epoch 15: Test loss 0.09969394457131378
Epoch 16: Test loss 0.09858253326203663
Epoch 17: Test loss 0.09748576206212958
Epoch 18: Test loss 0.09640354022596163
Epoch 19: Test loss 0.09533577104087777
Epoch 20: Test loss 0.09428235217681277
Epoch 21: Test loss 0.09324317603000401
Epoch 22: Test loss 0.09221813006013532
Epoch 23: Test loss 0.09120709712031555
Epoch 24: Test loss 0.09020995577936564
Epoch 25: Test